# 货车GPS数据


## 数据清理流程

In [182]:
import pandas as pd

# 设置输入和输出文件路径
input_file = 'E://20170906.csv'
output_file = './sample1.csv'

# 设置要读取的行数
n = 5000000 - 1

# 读取CSV文件的前n行
df = pd.read_csv(input_file, nrows=n, header=None)
# df = pd.read_csv(input_file, header=None)
# n  = len(df)
# df.to_parquet('E://20170906.parquet', index=False)
# 将前n行保存到新CSV文件
# 前 67489999 行已成功保存为 ./sample1.csv .

In [183]:
df = df[(df[2].between(115.41, 117.51)) & (df[3].between(39.44, 41.06))]
len(df)

3818463

In [184]:
df = df.drop_duplicates()
len(df)

3673179

In [185]:
nunique = df.groupby(df.columns[0])[df.columns[6]].nunique()
nunique.value_counts(2)

6
1    0.956324
2    0.043676
Name: proportion, dtype: float64

In [186]:
df = df[df[1].str.startswith('2017-09-05') | df[1].str.startswith('2017-09-06')]
len(df)

3659562

In [187]:
df = df.iloc[:, [0, 1, 2, 3, 4, 7]].rename(columns={
    0: '车号',
    1: '时间',
    2: '经度',
    3: '纬度',
    4: '速度',
    7: '角度'
})
df

,车号,时间,经度,纬度,速度,角度
0,80c3390a-77d1-4d1e-ad9c-6857dbd31fed,2017-09-05 23:58:16,116.636650,39.692375,79,307
1,366abd34-28b8-45c0-bfdd-61cfddaeb8fe,2017-09-05 23:58:15,116.481271,39.863000,49,180
3,d4995584-dbb1-4685-aa55-1da4ce79c8e1,2017-09-05 23:58:05,116.270770,40.211871,0,0
4,a8227ac6-e0f6-42da-a45b-a767d389bc59,2017-09-05 23:58:16,117.367031,40.826500,76,258
5,8cb1a9a6-e627-4ac7-9aab-c69f041922e6,2017-09-05 23:58:15,117.244695,39.969346,0,90
...,...,...,...,...,...,...
4999994,bd18e5b8-a4e4-4a2b-ad18-e33cc0a551f2,2017-09-06 01:39:36,116.635488,40.105466,0,0
4999995,8a62967c-1ad0-4996-af2c-5f4269906e71,2017-09-06 01:39:35,116.619455,39.961783,0,20
4999996,52b9c1f2-dc16-487e-add5-e8fdb752bf9c,2017-09-06 01:39:36,116.556351,39.732600,0,80
4999997,5931ede0-02e9-454d-a19a-44dfac36dd5e,2017-09-06 01:39:36,116.612265,40.191116,0,0


## 聚集地计算

In [188]:
# df.to_csv(output_file, index=False)
# print(f"预处理数据已成功保存为 {output_file} .")

In [189]:
# import pandas as pd
# output_file = './sample1.csv'
# df = pd.read_csv(output_file, low_memory=False)

In [190]:
df.loc[:, '时间'] = pd.to_datetime(df['时间'])

In [191]:
df = df[df['速度'] <= 2]
len(df)

2581241

### Haversine公式

In [192]:
import math
# 计算两点之间的球面距离
def haversine_distance(lat1, lon1, lat2, lon2):
    # 将十进制度数转换为弧度
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # haversine公式
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371393  # 地球平均半径，单位为米
    return c * r   # 结果单位为米

### 寻找1km对应的经纬度差

In [193]:
north = [39.9152097,116.3912757]
east = [39.906217,116.40299870]
center = [39.906217, 116.3912757]
print(haversine_distance(center[0],center[1],north[0],north[1])-1000)
print(haversine_distance(center[0],center[1],east[0],east[1])-1000)
deltaLat = north[0] - center[0]
deltaLon = east[1] - center[1]
print(deltaLat,deltaLon)

0.004299060397329413
-2.971175115362712e-05
0.008992700000000298 0.01172300000000348


### 计算最密聚集地

In [194]:
import numpy as np
import pandas as pd

# 定义北京市范围
BEIJING_BOUNDARY = {
    "lat_min": 39.44,
    "lat_max": 41.06,
    "lon_min": 115.41,
    "lon_max": 117.51,
}

# 定义近似正方形单位（1平方公里）
GRID_SIZE_LAT = 0.0089927 * 2 # 纬度差
GRID_SIZE_LON = 0.011723  * 2# 经度差


def create_grid(boundary, grid_size_lat, grid_size_lon):
    """创建北京市划分网格."""
    lat_range = np.arange(boundary["lat_min"], boundary["lat_max"], grid_size_lat)
    lon_range = np.arange(boundary["lon_min"], boundary["lon_max"], grid_size_lon)
    return lat_range, lon_range

def assign_points_to_grid(lat_range, lon_range, window=10):
    """将点分配到网格."""
    grid_counts = np.zeros((len(lat_range) - 2, len(lon_range) - 2))

    df['lat_idx'] = np.searchsorted(lat_range, df["纬度"], side="right") - 1
    df['lon_idx'] = np.searchsorted(lon_range, df["经度"], side="right") - 1
    df['valid'] = False
    df['outTime'] = False

    grouped = df.groupby('车号')
    for idxs, (car_id, group) in enumerate(grouped):
        group = group.sort_values('时间') 
        pastList = []
        futureList = [(a, b) for a, b in group.iloc[:min(window, len(group))][['lat_idx', 'lon_idx']].values]
        for num, (rowidx, row) in enumerate(group.iterrows()):
            lat_idx = row['lat_idx']
            lon_idx = row['lon_idx']            
            # 检查当前数据的lat_idx和lon_idx是否与未来二十条数据均不同，不同说明车辆即将离开当前网格
            futureList.pop(0)
            if num+window < len(group):
                futureList.append(tuple(group.iloc[num+window][['lat_idx', 'lon_idx']].values))
            if (lat_idx, lon_idx) not in futureList:
                df.loc[rowidx, 'outTime'] = True            
            # 检查当前数据的lat_idx和lon_idx是否与过去二十条数据均不同，不同说明刚刚进入当前网格
            pastList.append((lat_idx, lon_idx))
            if (lat_idx, lon_idx) not in pastList[:-1]:                
                df.loc[rowidx, 'valid'] = True
                if 0 <= lat_idx < grid_counts.shape[0] and 0 <= lon_idx < grid_counts.shape[1]:
                    grid_counts[lat_idx, lon_idx] += 1
                if 0 <= lat_idx - 1 < grid_counts.shape[0] and 0 <= lon_idx - 1 < grid_counts.shape[1]:
                    grid_counts[lat_idx - 1, lon_idx - 1] += 1
                if 0 <= lat_idx < grid_counts.shape[0] and 0 <= lon_idx - 1 < grid_counts.shape[1]:
                    grid_counts[lat_idx, lon_idx - 1] += 1
                if 0 <= lat_idx - 1 < grid_counts.shape[0] and 0 <= lon_idx < grid_counts.shape[1]:
                    grid_counts[lat_idx - 1, lon_idx] += 1
            if num >= window:
                pastList.pop(0)
    return grid_counts

In [195]:
lat_range, lon_range = create_grid(BEIJING_BOUNDARY, GRID_SIZE_LAT, GRID_SIZE_LON)
grid_counts = assign_points_to_grid(lat_range, lon_range)

C:\Users\sillybear\AppData\Local\Temp\ipykernel_10476\243456376.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat_idx'] = np.searchsorted(lat_range, df["纬度"], side="right") - 1
C:\Users\sillybear\AppData\Local\Temp\ipykernel_10476\243456376.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lon_idx'] = np.searchsorted(lon_range, df["经度"], side="right") - 1
C:\Users\sillybear\AppData\Local\Temp\ipykernel_10476\243456376.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of

#### 找参考区域和阈值

获取前20个4km * 4km大区域的阈值threshold

In [196]:
# 将二维数组展平为一维数组
flat_grid = grid_counts.flatten()

# 获取前100个最大值及其索引
top_100_indices = np.argpartition(flat_grid, -1000)[-1000:]
top_100_values = flat_grid[top_100_indices]

# 获取对应的二维索引
top_100_coords = np.unravel_index(top_100_indices, grid_counts.shape)

# 创建 DataFrame 以便更好地查看数据
df_top_100 = pd.DataFrame({
    'Value': top_100_values,
    'Row': top_100_coords[0],
    'Column': top_100_coords[1]
})

# 按值排序
df_top_100 = df_top_100.sort_values(by='Value', ascending=False).reset_index(drop=True)

print(df_top_100)

threshold = df_top_100['Value'][19]
threshold

      Value  Row  Column
0    1383.0   20      38
1    1382.0   20      39
2    1353.0   19      38
3    1309.0   19      39
4     966.0   37      49
..      ...  ...     ...
995    92.0   48      57
996    92.0   29      47
997    92.0   57      18
998    92.0   35      40
999    92.0   36      59

[1000 rows x 3 columns]


734.0

获取前20名的大区域

In [197]:
dense_regions = df_top_100[df_top_100['Value'] >= threshold][['Row', 'Column']].values.tolist()
dense_regions

[[20, 38],
 [20, 39],
 [19, 38],
 [19, 39],
 [37, 49],
 [21, 49],
 [16, 48],
 [37, 50],
 [16, 49],
 [22, 49],
 [19, 47],
 [17, 49],
 [49, 25],
 [17, 48],
 [37, 35],
 [34, 48],
 [38, 49],
 [13, 38],
 [37, 48],
 [19, 46]]

#### 在参考区域中寻找最密小区域和阈值

In [198]:
def findSubDense(regions:list, threshold = np.iinfo(np.int32).max):
    """
    寻找regions列表中每个元素所在区域(4km * 4km)的最密子区域(2km * 2km)
    返回最密子区域的列表和点最少的子区域所含的点
    """
    minThreshold = threshold
    subDense = [] # 存储最密子区域的经纬度范围
    for latidx, lonidx in regions:
        maxCount = 0        
        bound = (lat_range[latidx], lat_range[latidx+ 2], lon_range[lonidx], lon_range[lonidx + 2])
        dfRegion = df[(df['经度'].between(bound[2], bound[3])) & (df['纬度'].between(bound[0], bound[1]))]
        # return dfRegion, 0
        for ilat in range(11):
            for ilon in range(11):
                lon = bound[2] + GRID_SIZE_LON/10 * ilon
                lat = bound[0] + GRID_SIZE_LAT/10 * ilat
                count = len(dfRegion[
                    (dfRegion['经度'].between(lon, lon + GRID_SIZE_LON)) &
                    (dfRegion['纬度'].between(lat, lat + GRID_SIZE_LAT)) &
                    (dfRegion['valid'] == True)
                ])
                if maxCount < count:
                    maxCount = count
                    latRegion = lat
                    lonRegion = lon
        same = True
        for point in subDense:
            existing_lat, existing_lon, _ = point
            if abs(latRegion - existing_lat) < GRID_SIZE_LAT and abs(lonRegion - existing_lon) < GRID_SIZE_LON:
                same = False
                break
        if same:
            subDense.append([latRegion, lonRegion, maxCount])
            minThreshold = min(minThreshold, maxCount)
        # print(f"最密集区域的数据点数量: {maxCount}")
        # print(f"最密集区域的纬度范围: ({latRegion}, {latRegion + GRID_SIZE_LAT})")
        # print(f"最密集区域的经度范围: ({lonRegion}, {lonRegion + GRID_SIZE_LON})")
    print(f"最小阈值: {minThreshold}")
    subDense = pd.DataFrame(subDense, columns=["纬度", "经度", "计数"])
    return subDense, minThreshold

In [199]:
subDense, minThreshold = findSubDense(dense_regions, threshold)

最小阈值: 327


#### 找备选区域

In [200]:
reference_regions = df_top_100[df_top_100['Value'] >= minThreshold][['Row', 'Column']].values.tolist()
reference_regions

[[20, 38],
 [20, 39],
 [19, 38],
 [19, 39],
 [37, 49],
 [21, 49],
 [16, 48],
 [37, 50],
 [16, 49],
 [22, 49],
 [19, 47],
 [17, 49],
 [49, 25],
 [17, 48],
 [37, 35],
 [34, 48],
 [38, 49],
 [13, 38],
 [37, 48],
 [19, 46],
 [20, 49],
 [20, 46],
 [14, 39],
 [13, 39],
 [20, 47],
 [42, 33],
 [15, 42],
 [15, 47],
 [50, 24],
 [22, 48],
 [22, 34],
 [22, 54],
 [16, 47],
 [36, 50],
 [21, 50],
 [15, 48],
 [23, 49],
 [35, 52],
 [36, 35],
 [41, 33],
 [22, 33],
 [36, 36],
 [16, 42],
 [16, 46],
 [38, 50],
 [12, 38],
 [18, 38],
 [36, 49],
 [35, 48],
 [14, 38],
 [21, 48],
 [14, 42],
 [14, 41],
 [23, 54],
 [16, 50],
 [23, 48],
 [22, 50],
 [38, 48],
 [22, 53],
 [18, 39],
 [36, 48],
 [41, 34],
 [25, 55],
 [36, 51],
 [31, 39],
 [34, 52],
 [15, 43],
 [20, 50],
 [31, 47],
 [49, 24],
 [33, 48],
 [34, 49],
 [49, 30],
 [22, 55],
 [20, 57],
 [15, 41],
 [17, 39],
 [35, 51],
 [37, 36],
 [13, 41],
 [23, 53],
 [12, 37],
 [13, 42],
 [21, 38],
 [16, 41],
 [50, 20],
 [17, 42],
 [17, 38],
 [35, 53],
 [31, 46],
 [27, 49],

#### 在备选区域中找最密小区域

In [201]:
subDense, minThreshold = findSubDense(reference_regions)
subDense.sort_values(by = '计数', ascending = False, inplace=True)
subDense

最小阈值: 137


,纬度,经度,计数
0,39.801507,116.322049,1034
7,40.333874,116.005528,706
2,39.824888,116.575266,538
6,39.790715,116.511962,521
3,39.745752,116.554165,491
...,...,...,...
49,39.993950,116.558854,148
50,39.817693,116.488516,147
58,39.889635,116.162617,144
54,39.691796,116.364252,141


<!-- 参考区域数量: 1
阈值：929452.1989
纬度范围: (39.799708, 39.817693), 经度范围: (116.324394, 116.347840), 点个数：930855

最密集区域的数据点数量: 3650
最密集区域的纬度范围: (39.80600289000001, 39.81499559000001)
最密集区域的经度范围: (116.33260010000026, 116.34432310000027)

备选区域数量: 3
阈值：452968.0
纬度范围: (39.799708, 39.8176934), 经度范围: (116.312671, 116.336117), 点个数：729496
纬度范围: (39.799708, 39.8176934), 经度范围: (116.324394, 116.347840), 点个数：930855
纬度范围: (39.808701, 39.826686), 经度范围: (116.324394, 116.347840), 点个数：666237

最密集区域的数据点数量: 414776
最密集区域的纬度范围: (39.802406, 39.811399)
最密集区域的经度范围: (116.324394, 116.336117)
最密集区域的数据点数量: 452968
最密集区域的纬度范围: (39.80600289000001, 39.81499559000001)
最密集区域的经度范围: (116.33260010000026, 116.34432310000027)
最密集区域的数据点数量: 429662
最密集区域的纬度范围: (39.808700, 39.817693)
最密集区域的经度范围: (116.330256, 116.341979) -->

#### 获取前20最密聚集地

In [202]:
Top20 = subDense[:20]
print(f"阈值：{Top20['计数'].min()}")
print(Top20)

阈值：327
           纬度          经度    计数
0   39.801507  116.322049  1034
7   40.333874  116.005528   706
2   39.824888  116.575266   538
6   39.790715  116.511962   521
3   39.745752  116.554165   491
10  39.688199  116.322049   483
11  40.195387  116.202475   447
31  40.355457  115.883609   444
28  39.814096  116.755800   440
14  39.846470  116.214198   402
1   40.114453  116.568232   397
9   40.062295  116.558854   391
8   40.112654  116.242333   371
5   39.850067  116.572922   369
15  39.851866  116.683118   366
27  40.333874  116.113380   354
23  39.896829  116.708908   338
57  40.369845  116.075866   328
24  40.006540  116.333772   328
4   40.110855  116.601057   327


In [203]:
round = 3
print(Top20[['纬度', '经度', '计数']].to_latex(index=False, float_format=f"%.{round}f"))

\begin{tabular}{rrr}
\toprule
纬度 & 经度 & 计数 \\
\midrule
39.802 & 116.322 & 1034 \\
40.334 & 116.006 & 706 \\
39.825 & 116.575 & 538 \\
39.791 & 116.512 & 521 \\
39.746 & 116.554 & 491 \\
39.688 & 116.322 & 483 \\
40.195 & 116.202 & 447 \\
40.355 & 115.884 & 444 \\
39.814 & 116.756 & 440 \\
39.846 & 116.214 & 402 \\
40.114 & 116.568 & 397 \\
40.062 & 116.559 & 391 \\
40.113 & 116.242 & 371 \\
39.850 & 116.573 & 369 \\
39.852 & 116.683 & 366 \\
40.334 & 116.113 & 354 \\
39.897 & 116.709 & 338 \\
40.370 & 116.076 & 328 \\
40.007 & 116.334 & 328 \\
40.111 & 116.601 & 327 \\
\bottomrule
\end{tabular}



### ODIO分析：针对区域中每条数据计算进入时间、离开时间、出发地、目的地

In [204]:
def odAnalysis(regions, timeThreshold = 600, lat_range = lat_range, lon_range = lon_range):
    """
    对regions中的每个区域进行OD分析
    """
    allOrigin = []
    allIntime = []
    allDest = []
    allOuttime = []
    for lat, lon, _ in regions:
        origin = []
        dest = []
        inTime = []
        outTime = []
        bound = (lat, lat + GRID_SIZE_LAT, lon, lon + GRID_SIZE_LON)
        dfRegion = df[(df['经度'].between(bound[2], bound[3])) & (df['纬度'].between(bound[0], bound[1]))]
        grouped = dfRegion.groupby('车号')
        for car_id, group in grouped:
            group = group.sort_values(by='时间')
            for idx, row in group.iterrows():
                if row['outTime']:
                    outTime.append(row['时间'])
                if not row['valid'] :
                    continue                
                inTime.append(row['时间'])
                def findStop(stop, arrive, types):
                    """找到最近的连续600秒停车的记录"""
                    nowlon = row['lon_idx']
                    nowlat = row['lat_idx']
                    for _, rowStop in stop.iterrows():
                        # 静止
                        # if rowStop['速度'] < 2:
                            # 静止时间超过timeThreshold
                            # 且车辆位置发生变化
                            if (arrive - rowStop['时间']).total_seconds() > timeThreshold and \
                                (rowStop['lon_idx'] != row['lon_idx'] or rowStop['lat_idx'] != row['lat_idx']) and \
                                (rowStop['lon_idx'] == nowlon and rowStop['lat_idx'] == nowlat): 
                                # print(f"车号: {rowStop['车号']}{types}信息：\n \
                                #     由 {rowStop['时间']}, 纬度: {rowStop['纬度']}, 经度: {rowStop['经度']}\n \
                                #     到 {row['时间']}, 纬度: {row['纬度']}, 经度: {row['经度']}")
                                lat_idx = np.searchsorted(lat_range, rowStop['纬度'], side="right") - 1
                                lon_idx = np.searchsorted(lon_range, rowStop['经度'], side="right") - 1
                                return [lat_idx, lon_idx]
                            else:
                                arrive = rowStop['时间']
                                nowlon = rowStop['lon_idx']
                                nowlat = rowStop['lat_idx']
                        # else:
                        #     arrive = rowStop['时间']
                    return None
                before_stop = group.loc[:idx]
                arrive = row['时间']
                stop = findStop(before_stop[::-1], arrive, "进入")
                if stop is not None:
                    origin.append(stop)
                
                # 找到之后最近的连续600秒停车的记录
                after_stop = group.loc[idx:]
                stop = findStop(after_stop, arrive, "离开")
                if stop is not None:
                    dest.append(stop)
    
        print(f"区域({lat}, {lon})的OD分析结果：")
        print(f"进入区域的车辆数：{len(origin)}")
        print(f"离开区域的车辆数：{len(dest)}")
        print(f"进入区域的车辆信息：{origin}")
        print(f"离开区域的车辆信息：{dest}")
        print(f"进入区域的时间：{inTime}")
        print(f"离开区域的时间：{outTime}")
        allOrigin.append(origin)
        allDest.append(dest)
        allIntime.append(inTime)
        allOuttime.append(outTime)
    return allOrigin, allDest, allIntime, allOuttime

In [205]:
allOrigin, allDest, allIntime, allOuttime = odAnalysis(Top20[['纬度', '经度', '计数']].values, 60)
# 每一个都是二维列表，第一维是区域，第二维是具体数据
# allOrigin和allDest的每个元素是一个列表，列表中的每个元素是一个列表，包含纬度、经度
# allIntime和allOuttime的每个元素是一个列表，列表中的每个元素是一个时间戳

区域(39.80150654000001, 116.32204940000027)的OD分析结果：
进入区域的车辆数：11
离开区域的车辆数：0
进入区域的车辆信息：[[20, 39], [20, 39], [20, 39], [21, 39], [20, 39], [20, 38], [20, 39], [21, 39], [20, 39], [20, 39], [20, 38]]
离开区域的车辆信息：[]
进入区域的时间：[Timestamp('2017-09-06 00:23:00'), Timestamp('2017-09-05 23:58:35'), Timestamp('2017-09-05 23:56:16'), Timestamp('2017-09-05 23:59:12'), Timestamp('2017-09-06 00:03:18'), Timestamp('2017-09-05 23:58:51'), Timestamp('2017-09-05 23:58:58'), Timestamp('2017-09-06 00:02:21'), Timestamp('2017-09-05 23:59:24'), Timestamp('2017-09-06 00:34:08'), Timestamp('2017-09-06 01:03:29'), Timestamp('2017-09-06 00:34:43'), Timestamp('2017-09-06 00:05:32'), Timestamp('2017-09-06 00:07:51'), Timestamp('2017-09-06 00:01:12'), Timestamp('2017-09-05 23:59:14'), Timestamp('2017-09-06 00:01:28'), Timestamp('2017-09-05 23:59:19'), Timestamp('2017-09-06 01:29:15'), Timestamp('2017-09-06 00:05:32'), Timestamp('2017-09-06 00:01:24'), Timestamp('2017-09-06 00:24:54'), Timestamp('2017-09-06 00:00:00'), Ti

In [206]:
import os
import pickle

output_dir = 'output'

# 定义文件路径
allOrigin_path = os.path.join(output_dir, 'allOrigin.pkl')
allDest_path = os.path.join(output_dir, 'allDest.pkl')
allIntime_path = os.path.join(output_dir, 'allIntime.pkl')
allOuttime_path = os.path.join(output_dir, 'allOuttime.pkl')

# 将结果存储到文件中
with open(allOrigin_path, 'wb') as f:
    pickle.dump(allOrigin, f)

with open(allDest_path, 'wb') as f:
    pickle.dump(allDest, f)

with open(allIntime_path, 'wb') as f:
    pickle.dump(allIntime, f)

with open(allOuttime_path, 'wb') as f:
    pickle.dump(allOuttime, f)

print("Results have been saved to the output folder.")

Results have been saved to the output folder.


In [207]:
with open(allOrigin_path, 'rb') as f:
    allOrigin = pickle.load(f)
with open(allDest_path, 'rb') as f:
    allDest = pickle.load(f)
with open(allIntime_path, 'rb') as f:
    allIntime = pickle.load(f)
with open(allOuttime_path, 'rb') as f:
    allOuttime = pickle.load(f)

### 可视化

#### 可视化前20最密聚集地

In [233]:
import folium

def map2html(data, name = 'map.html'):
    # 创建地图对象，设置初始位置和缩放级别
    m = folium.Map([39.908722, 116.397499], zoom_start=12)

    # 添加20个密集区域的标记
    i = 1
    for lat, lon, cnt in data[['纬度', '经度', '计数']].values:
        folium.Polygon([
                [lat, lon],
                [lat, lon + GRID_SIZE_LON],
                [lat + GRID_SIZE_LAT, lon + GRID_SIZE_LON],
                [lat + GRID_SIZE_LAT, lon],
                [lat, lon],
            ],
            color='#FF0000',  # 多边形边界颜色
            fill=False
        ).add_to(m)
        # 在多边形左上角添加标记
        folium.Marker(
            [lat + GRID_SIZE_LAT * 5 / 10, lon + GRID_SIZE_LON * 15 / 10],
            icon=folium.DivIcon(html=f'''
                <div class ="my-label" style="font-size: 10pt; color: red; white-space: nowrap;">
                    第{i}聚集地, 共{int(cnt)}辆次车停留<br>经纬度位置为：({lat:.3f},{lon:.3f})
                </div>
            ''')
        ).add_to(m)
        i += 1
    # 调整代码以适应不同的分辨率
    # map_id = m.get_name()
    # m.get_root().html.add_child(folium.Element(f'''
    #     <script>
    #     function updateLabelSize() {{
    #         var map = "{map_id}";  // 使用Folium生成的地图ID
    #         var zoom = map.getZoom();
    #         var labels = document.querySelectorAll('.my-label');
    #         labels.forEach(function(label) {{
    #             label.style.fontSize = (zoom * 1.5) + 'pt';
    #         }});
    #     }}
    #     map.on('zoomend', updateLabelSize);
    #     updateLabelSize();
    #     </script>
    # '''))
    # 保存地图到HTML文件
    m.save('./html/' + name)
    

map2html(Top20, 'Top20.html')
# map2html2(dfResult)
# 北野场桥 = dfv[(dfv['经度'].between(116.425, 116.45)) & (dfv['纬度'].between(39.715, 39.73))]
# map2html(北野场桥, '北野场桥.html')

#### 可视化进入和离开时间：每个最密聚集地和所有最密聚集地的和

In [217]:
import matplotlib.pyplot as plt
from collections import Counter

plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

def remove_outliers(data, threshold=1.5):
    """使用IQR方法移除离群值."""
    data = np.array(data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - threshold * iqr
    upper_bound = q3 + threshold * iqr
    return data[(data >= lower_bound) & (data <= upper_bound)]

def InOutAnalysis(intime, outtime, name, i = -1):
    # 移除进入时间和离开时间中的离群值
    intime = remove_outliers(intime)
    outtime = remove_outliers(outtime)
    
    plt.figure(figsize=(10, 5))
    plt.hist(intime, bins=24, alpha=0.5, label='进入时间')
    plt.hist(outtime, bins=24, alpha=0.5, label='离开时间')
    plt.title(f'{name} 的进入时间和离开时间分布')
    plt.xlabel('时间')
    plt.ylabel('频率')
    plt.legend()
    plt.tight_layout()  # 调整布局以防止标签重叠
    if i == -1:
        plt.savefig(f"img/时序分布/所有区域时间分布.png")
    else:
        plt.savefig(f'img/时序分布/区域{i}时间分布.png')
    plt.close()


for i, (idx, row) in enumerate(Top20.iterrows()):
    lat = row['纬度']
    lon = row['经度']
    InOutAnalysis(allIntime[i], allOuttime[i], f"区域{i+1}:({lat:.3f}, {lon:.3f})", i+1)


In [218]:
# 分析所有区域的进入时间、离开时间分布
all_intime = [time for sublist in allIntime for time in sublist]
all_outtime = [time for sublist in allOuttime for time in sublist]
InOutAnalysis(all_intime, all_outtime, '所有区域')

#### 可视化OD分布：每个最密聚集地和所有最密聚集地的和

In [224]:
def ODAnalysis(origin, dest, name, i = -1):
    # 统计出发地和目的地的频率
    origin_counter = Counter((lat, lon) for lat, lon in origin)
    dest_counter = Counter((lat, lon) for lat, lon in dest)

    # 取前20名
    top_20_origins = origin_counter.most_common(20)
    top_20_dests = dest_counter.most_common(20)

    # 分别获取纬度、经度和计数
    if top_20_origins:
        origin_lats, origin_lons, origin_counts = zip(*[(lat, lon, count) for (lat, lon), count in top_20_origins])
        origin_labels = [(lat, lon) for lat, lon in zip(origin_lats, origin_lons)]
    else:
        origin_counts = []
        origin_labels = []

    if top_20_dests:
        dest_lats, dest_lons, dest_counts = zip(*[(lat, lon, count) for (lat, lon), count in top_20_dests])
        dest_labels = [(lat, lon) for lat, lon in zip(dest_lats, dest_lons)]
    else:
        dest_counts = []
        dest_labels = []

    # 合并标签
    all_labels = list(set(origin_labels + dest_labels))
    all_labels.sort()

    # 创建标签到索引的映射
    label_to_index = {label: idx for idx, label in enumerate(all_labels)}

    # 获取出发地和目的地的索引
    origin_indices = [label_to_index[label] for label in origin_labels]
    dest_indices = [label_to_index[label] for label in dest_labels]

    # 转换标签为字符串格式
    formatted_labels = [f'({(lat_range[lat] + lat_range[lat+1])/2:.4f},\n{(lon_range[lon] + lon_range[lon+1])/2:.4f})' for lat, lon in all_labels]

    # 绘制图表
    plt.figure(figsize=(10, 5))
    if origin_counts:
        plt.bar(origin_indices, origin_counts, alpha=0.5, label='出发地', color='blue')
    if dest_counts:
        plt.bar(dest_indices, dest_counts, alpha=0.5, label='目的地', color='orange')

    # 设置横轴标签并设置标签字号
    plt.xticks(range(len(all_labels)), formatted_labels, rotation=0, fontsize=10)

    plt.title(f'{name} 的出发地和目的地分布')
    plt.xlabel('区域坐标')
    plt.ylabel('频率')
    plt.legend()
    plt.tight_layout()  # 调整布局以防止标签重叠
    if i == -1:
        plt.savefig(f"img/OD分布/所有区域出发地和目的地分布.png")
        plt.xticks(range(len(all_labels)), formatted_labels, rotation=0, fontsize=8)
    else:
        plt.savefig(f"img/OD分布/区域{i}出发地和目的地分布.png")
    plt.close()
    return origin_counts, origin_labels, dest_counts, dest_labels

for i, (idx,row) in enumerate(Top20.iterrows()):
    lat = row['纬度']
    lon = row['经度']
    ODAnalysis(allOrigin[i], allDest[i], f"区域{i+1}:({lat:.3f}, {lon:.3f})", i+1)
    # break

In [225]:
all_origins = [item for sublist in allOrigin for item in sublist]
all_dests = [item for sublist in allDest for item in sublist]
origin_counts, origin_labels, dest_counts, dest_labels = ODAnalysis(all_origins, all_dests, "Top20")


#### 可视化最密聚集地和OD

In [226]:
def map2html2(data, name = 'map.html', od = False):
    # 创建地图对象，设置初始位置和缩放级别
    m = folium.Map([39.908722, 116.397499], zoom_start=12)

    # 添加20个密集区域的标记
    i = 1
    for lat, lon in data[['纬度', '经度']].values:
        folium.Polygon([
            [lat, lon],
            [lat, lon + GRID_SIZE_LON],
            [lat + GRID_SIZE_LAT, lon + GRID_SIZE_LON],
            [lat + GRID_SIZE_LAT, lon],
            [lat, lon],
        ],
        color='#FF0000',  # 多边形边界颜色
        fill=False
    ).add_to(m)
        # 在多边形左上角添加标记
        folium.Marker(
            [lat + GRID_SIZE_LAT * 9.5 / 10, lon + GRID_SIZE_LON * 0.5 / 10],
            icon=folium.DivIcon(html=f'''
                <div class ="my-label" style="font-size: 10pt; color: red; white-space: nowrap;">
                    第{i}聚集地
                </div>
            ''')
        ).add_to(m)
        i += 1
    if od:
        for i, origin in enumerate(origin_labels):
            lat, lon = origin[0], origin[1]
            folium.Marker(
                [lat_range[lat] + GRID_SIZE_LAT / 2, lon_range[lon] + GRID_SIZE_LON / 2],
                icon=folium.Icon(color='green', icon='info-sign')
            ).add_to(m)
        for i, dest in enumerate(dest_labels):
            lat, lon = dest[0], dest[1]
            folium.Marker(
                [lat_range[lat] + GRID_SIZE_LAT / 2, lon_range[lon] + GRID_SIZE_LON / 2],
                icon=folium.Icon(color='yellow', icon='info-sign')
            ).add_to(m)

    # 保存地图到HTML文件
    m.save('./html/' + name)

map2html2(Top20, 'Top20withOD.html', True)